# Sarmad Ali 
# **10 custom queries**

# Query Proposition 1

This query retrieves customers who placed an order in the last six months by checking recent orders in the `Sales.Orders` table.

## Use Case

**Scenario**:  
A marketing team wants to target customers who have made recent purchases. They run this query to identify customers who placed orders within the last six months.

**Action**:  
The query returns a list of `CustomerID` and `CustomerName`, which can be used for email campaigns or promotions.

**Outcome**:  
Focused marketing efforts on active customers, increasing the chances of repeat business.

### Benefits:
- **Targeted Marketing**: Engage only with recently active customers.
- **Customer Retention**: Reach out to recent buyers to encourage loyalty.
- **Timely Promotions**: Use recent activity data for time-sensitive campaigns.


In [ ]:
USE WideWorldImporters;
GO

SELECT 
    C.CustomerID, 
    C.CustomerName
FROM 
    Sales.Customers AS C
WHERE 
    C.CustomerID IN (
        SELECT DISTINCT O.CustomerID
        FROM Sales.Orders AS O
        WHERE O.OrderDate >= DATEADD(MONTH, -6, GETDATE())
    );


# Query Proposition 2

This query retrieves suppliers who offer more than 10 products by counting the number of stock items for each supplier.

## Use Case

**Scenario**:  
A procurement team wants to identify suppliers with a substantial inventory. They run this query to focus on suppliers who can provide a wide range of products.

**Action**:  
The query returns a list of `SupplierID` and `ProductCount` for suppliers with more than 10 products.

**Outcome**:  
The team can make informed decisions about which suppliers to engage for bulk purchasing or strategic partnerships.

### Benefits:

- **Supplier Evaluation**: Identify reliable suppliers with a diverse product range.
- **Inventory Management**: Focus on suppliers that can meet large order requirements.
- **Strategic Partnerships**: Build relationships with suppliers that offer extensive product lines.

In [ ]:
USE WideWorldImporters;
GO
WITH SupplierProducts AS (
    SELECT 
        SupplierID, 
        COUNT(StockItemID) AS ProductCount
    FROM 
        Warehouse.StockItems
    GROUP BY 
        SupplierID
)
SELECT 
    SupplierID, 
    ProductCount
FROM 
    SupplierProducts
WHERE 
    ProductCount > 10;

# Query Proposition 3

This query calculates the average order value for each customer by summing the total value of their orders.

## Use Case

**Scenario**:  
A sales team wants to assess customer spending habits to tailor marketing strategies. They run this query to find out how much, on average, customers spend per order.

**Action**:  
The query returns a list of `CustomerID` and their corresponding `AverageOrderValue`.

**Outcome**:  
The sales team can identify high-value customers and develop targeted promotions or loyalty programs.

### Benefits:

- **Customer Insights**: Understand customer spending patterns to improve service.
- **Targeted Marketing**: Create tailored campaigns for high-value customers.
- **Sales Strategy**: Identify opportunities for upselling and cross-selling based on order values.

In [ ]:
USE WideWorldImporters;
GO
WITH OrderValues AS (
    SELECT 
        O.CustomerID, 
        SUM(OL.Quantity * OL.UnitPrice) AS OrderValue
    FROM 
        Sales.Orders O
    JOIN 
        Sales.OrderLines OL ON O.OrderID = OL.OrderID
    GROUP BY 
        O.CustomerID, O.OrderID
)
SELECT 
    CustomerID, 
    AVG(OrderValue) AS AverageOrderValue
FROM 
    OrderValues
GROUP BY 
    CustomerID;

# Query Proposition 4

This query retrieves the top 5 stock items with the highest maximum prices from the order lines.

## Use Case

**Scenario**:  
A product management team wants to identify which stock items command the highest prices in orders. They run this query to analyze the top-performing products by price.

**Action**:  
The query returns a list of the top 5 `StockItemID`, `StockItemName`, and their corresponding `MaxPrice`.

**Outcome**:  
The product management team can focus on high-value items, assess pricing strategies, and make informed decisions on inventory management and marketing efforts.

### Benefits:

- **Market Analysis**: Understand which products are perceived as premium and priced higher.
- **Pricing Strategy**: Adjust pricing based on market demand and competitive analysis.
- **Inventory Management**: Identify and promote high-value stock items to boost sales.

In [ ]:
USE WideWorldImporters;
GO
SELECT TOP 5
    SI.StockItemID, 
    SI.StockItemName, 
    MAX(OL.UnitPrice) AS MaxPrice
FROM 
    Sales.OrderLines OL
JOIN 
    Warehouse.StockItems SI ON OL.StockItemID = SI.StockItemID
GROUP BY 
    SI.StockItemID, SI.StockItemName
ORDER BY 
    MaxPrice DESC

# Query Proposition 5

This query retrieves a list of customers who have not placed any orders in the past year.

## Use Case

**Scenario**:  
A marketing team wants to identify inactive customers who haven’t made any purchases in the last year. This information is essential for targeted re-engagement campaigns.

**Action**:  
The query returns `CustomerID` and `CustomerName` for customers that do not have any orders recorded in the last 12 months.

**Outcome**:  
The marketing team can use this data to develop strategies aimed at re-engaging these customers, such as sending promotional offers or personalized communications.

### Benefits:

- **Customer Retention**: Identify customers who may need re-engagement to prevent churn.
- **Targeted Campaigns**: Develop tailored marketing campaigns to encourage lapsed customers to return.
- **Sales Insights**: Gain insights into customer behavior and purchase patterns over the past year.

In [ ]:
USE WideWorldImporters;
GO
SELECT 
    C.CustomerID, 
    C.CustomerName
FROM 
    Sales.Customers AS C
WHERE 
    C.CustomerID NOT IN (
        SELECT CustomerID
        FROM Sales.Orders
        WHERE OrderDate >= DATEADD(YEAR, -1, GETDATE())
    );


# Query Proposition 6

This query identifies products that have been ordered more than 100 times and retrieves their details.

## Use Case

**Scenario**:  
A sales analysis team wants to focus on high-demand products in the inventory. By identifying these products, they can ensure adequate stock levels and plan for future procurement.

**Action**:  
The query retrieves `StockItemID`, `StockItemName`, and `OrderCount` for items that have been ordered over 100 times.

**Outcome**:  
The sales team can prioritize these high-selling products in their inventory management and promotional strategies.

### Benefits:

- **Inventory Management**: Helps maintain stock levels for popular items to meet customer demand.
- **Sales Strategies**: Allows the sales team to create targeted promotions for high-demand products.
- **Business Insights**: Provides insights into customer purchasing behavior and product popularity.

In [ ]:
USE WideWorldImporters;
GO
WITH ProductOrders AS (
    SELECT 
        StockItemID, 
        COUNT(OrderID) AS OrderCount
    FROM 
        Sales.OrderLines
    GROUP BY 
        StockItemID
)
SELECT 
    SI.StockItemID, 
    SI.StockItemName, 
    PO.OrderCount
FROM 
    Warehouse.StockItems SI
JOIN 
    ProductOrders PO ON SI.StockItemID = PO.StockItemID
WHERE 
    PO.OrderCount > 100;

# Query Proposition 7

This query calculates the total revenue generated from each customer and retrieves their details along with the revenue amounts.

## Use Case

**Scenario**:  
A business wants to analyze its revenue contribution from different customers to identify high-value clients. Understanding customer revenue helps in strategizing marketing efforts and improving customer relationships.

**Action**:  
The query calculates the total revenue (`Revenue`) for each customer by summing the product of quantity and unit price from their orders. It then retrieves the `CustomerID`, `CustomerName`, and total `Revenue`, sorting the results in descending order.

**Outcome**:  
The business can identify top customers based on their revenue contribution, enabling targeted communication and personalized offers.

### Benefits:

- **Customer Segmentation**: Helps identify high-value customers for loyalty programs.
- **Revenue Insights**: Provides a clear view of revenue distribution across the customer base.
- **Strategic Planning**: Assists in aligning marketing and sales strategies based on customer value.

In [ ]:
USE WideWorldImporters;
GO
WITH CustomerRevenue AS (
    SELECT 
        O.CustomerID, 
        SUM(OL.Quantity * OL.UnitPrice) AS Revenue
    FROM 
        Sales.Orders O
    JOIN 
        Sales.OrderLines OL ON O.OrderID = OL.OrderID
    GROUP BY 
        O.CustomerID
)
SELECT 
    C.CustomerID, 
    C.CustomerName, 
    CR.Revenue
FROM 
    Sales.Customers C
JOIN 
    CustomerRevenue CR ON C.CustomerID = CR.CustomerID
ORDER BY 
    CR.Revenue DESC;

# Query Proposition 8

This query retrieves the last order date for each stock item in the inventory, providing insight into the most recent sales activity for each product.

## Use Case

**Scenario**:  
A retail manager wants to track the most recent sales for each stock item to monitor product performance and inventory turnover. Knowing when each item was last ordered can help in inventory management and restocking decisions.

**Action**:  
The query selects the `StockItemID` and `StockItemName` from the `Warehouse.StockItems` table. It uses a subquery to find the maximum `OrderDate` from the `Sales.OrderLines` and `Sales.Orders` tables, filtered by `StockItemID`. This gives the date of the last order for each stock item.

**Outcome**:  
The retail manager can quickly assess which items are actively being sold and which may need reordering or promotions to boost sales.

### Benefits:

- **Inventory Management**: Assists in identifying fast-moving and slow-moving items.
- **Sales Insights**: Provides a clear view of recent sales trends for each product.
- **Informed Decision-Making**: Enables better planning for inventory restocking and promotions.

In [ ]:
USE WideWorldImporters;
GO
SELECT 
    SI.StockItemID, 
    SI.StockItemName, 
    (SELECT MAX(O.OrderDate)
     FROM Sales.OrderLines OL
     JOIN Sales.Orders O ON OL.OrderID = O.OrderID
     WHERE OL.StockItemID = SI.StockItemID
    ) AS LastOrderDate
FROM 
    Warehouse.StockItems SI;

# Query Proposition 9

This query identifies suppliers who offer more than five products priced above the average unit price in the inventory.

## Use Case

**Scenario**:  
A procurement manager wants to evaluate suppliers based on their product pricing to identify those who offer a substantial number of higher-priced items. This information can be useful for negotiating contracts and understanding supplier offerings.

**Action**:  
The query first calculates the average unit price of all stock items in the `Warehouse.StockItems` table. It then counts the number of products for each supplier where the unit price exceeds this average. Finally, it filters the results to include only those suppliers with more than five above-average products.

**Outcome**:  
The procurement manager can quickly pinpoint which suppliers have a significant portfolio of higher-priced items, helping in supplier evaluation and negotiation strategies.

### Benefits:

- **Supplier Evaluation**: Helps assess suppliers based on their range of higher-value products.
- **Cost Analysis**: Provides insights into pricing strategies across different suppliers.
- **Informed Decision-Making**: Aids in selecting suppliers that align with the company's pricing and product strategies.

In [ ]:
USE WideWorldImporters;
GO
WITH AboveAverageProducts AS (
    SELECT 
        SupplierID, 
        COUNT(StockItemID) AS ProductCount
    FROM 
        Warehouse.StockItems
    WHERE 
        UnitPrice > (SELECT AVG(UnitPrice) FROM Warehouse.StockItems)
    GROUP BY 
        SupplierID
)
SELECT 
    SupplierID, 
    ProductCount
FROM 
    AboveAverageProducts
WHERE 
    ProductCount > 5;

# Query Proposition 10

This query retrieves orders that contain more than five distinct products, providing insights into larger purchases made by customers.

## Use Case

**Scenario**:  
A sales manager wants to identify customer orders with a substantial number of products. This information can help in analyzing customer buying patterns and may assist in targeted marketing strategies.

**Action**:  
The query calculates the count of distinct products associated with each order by grouping records from the `Sales.OrderLines` table. It then joins this count with the corresponding order details from the `Sales.Orders` table, filtering to show only those orders where the product count exceeds five.

**Outcome**:  
The sales manager can identify high-volume orders, which could indicate bulk purchasing behavior, allowing for tailored follow-up actions or promotional offers.

### Benefits:

- **Customer Insights**: Helps understand customer purchasing behavior based on order size.
- **Targeted Marketing**: Enables focused marketing efforts toward customers with high-volume orders.
- **Sales Strategy**: Aids in evaluating which products are frequently purchased together, potentially guiding inventory and sales strategies.

In [ ]:
USE WideWorldImporters;
GO
WITH OrderProductCount AS (
    SELECT 
        OrderID, 
        COUNT(DISTINCT StockItemID) AS ProductCount
    FROM 
        Sales.OrderLines
    GROUP BY 
        OrderID
)
SELECT 
    O.OrderID, 
    O.OrderDate, 
    OPC.ProductCount
FROM 
    Sales.Orders O
JOIN 
    OrderProductCount OPC ON O.OrderID = OPC.OrderID
WHERE 
    OPC.ProductCount > 5;